In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

from tqdm import tqdm
from thesis.utils.pytorch import loop_fn, evaluate, train

In [3]:
# Define data transforms
# MobileNetV2 expects 224x224 input images and specific normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),          # phóng to rồi crop ngẫu nhiên vùng 224x224
        transforms.RandomHorizontalFlip(p=0.5),     # lật ngang ảnh ngẫu nhiên
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # thay đổi màu sắc
        transforms.RandomRotation(degrees=15),      # xoay ngẫu nhiên ±15 độ
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # dịch ảnh
        transforms.RandomPerspective(distortion_scale=0.2, p=0.5), # hiệu ứng phối cảnh
        transforms.ToTensor(),                      # chuyển thành tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([ # Often same as validation for consistent evaluation
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


# base_dir = "/Users/anand/Desktop/1mg/repos/thesis/thesis/data/PlantDoc-Dataset"
base_dir = "/Users/anand/Desktop/self/thesis/data/PlantVillage"
base_dataset = datasets.ImageFolder(base_dir)

total_size = len(base_dataset)
print('Total size:', total_size)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

print('Train size:', train_size)
print('Val size:', val_size)
print('Test size:', test_size)

train_data, val_data, test_data = random_split(
    base_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

train_data.dataset.transform = data_transforms["train"]
val_data.dataset.transform = data_transforms["val"]
test_data.dataset.transform = data_transforms["test"]

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, shuffle=True, pin_memory=True)


Total size: 20638
Train size: 16510
Val size: 2063
Test size: 2065


In [ ]:
from collections import Counter

# These give you the label for each sample in the subset
train_labels = [label for _, label in train_data]
val_labels = [label for _, label in val_data]
test_labels = [label for _, label in test_data]

train_counters = Counter(train_labels)
val_counters =  Counter(val_labels)
test_counters = Counter(test_labels)






dict_keys([7, 6, 0, 1, 9, 12, 11, 14, 8, 5, 3, 2, 4, 13, 10])
Counter({12: 334, 5: 241, 7: 175, 9: 172, 10: 165, 14: 158, 1: 149, 11: 127, 2: 108, 0: 104, 8: 98, 3: 93, 6: 87, 13: 40, 4: 12})
Counter({12: 331, 5: 207, 7: 186, 9: 177, 10: 174, 1: 158, 14: 139, 11: 132, 3: 118, 6: 109, 2: 106, 8: 93, 0: 84, 13: 33, 4: 18})


In [49]:
x = list(train_counters.keys())
x.sort()
print(x)

y = list(val_counters.keys())
y.sort()
print(y)

z = list(test_counters.keys())
z.sort()
print(z)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [ ]:
print(train_loader)

In [13]:
class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, mlp_dim):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=dim, 
                nhead=heads, 
                dim_feedforward=mlp_dim,
                activation="gelu",
                batch_first=True
            )
            for _ in range(depth)
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


In [10]:
class MobileViTBlock(nn.Module):
    def __init__(self, in_channels, transformer_dim, ffn_dim, n_transformer_blocks, patch_size):
        super().__init__()
        self.local_rep = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, groups=in_channels),
            nn.BatchNorm2d(in_channels),
            nn.Conv2d(in_channels, transformer_dim, kernel_size=1),
            nn.BatchNorm2d(transformer_dim),
            nn.ReLU()
        )
        
        self.transformer = Transformer(
            dim=transformer_dim,
            depth=n_transformer_blocks,
            heads=4,
            mlp_dim=ffn_dim
        )

        self.fuse = nn.Sequential(
            nn.Conv2d(transformer_dim, in_channels, kernel_size=1),
            nn.BatchNorm2d(in_channels),
            nn.ReLU()
        )

        self.patch_size = patch_size

    def forward(self, x):
        y = self.local_rep(x)  # shape: [B, C, H, W]
        B, C, H, W = y.shape
        ph, pw = self.patch_size

        # Flatten patches
        y = y.unfold(2, ph, ph).unfold(3, pw, pw)  # shape: [B, C, H/ph, W/pw, ph, pw]
        y = y.contiguous().view(B, C, -1, ph * pw)  # [B, C, N, P]
        y = y.permute(0, 2, 1, 3).contiguous().view(B * y.shape[2], ph * pw, C)  # [B*N, P, C]

        y = self.transformer(y)  # [B*N, P, C]

        # Reshape back
        y = y.view(B, -1, ph * pw, C).permute(0, 3, 1, 2)  # [B, C, N, P]
        num_patches = y.shape[2]
        patches_per_row = W // pw
        y = y.view(B, C, H, W)

        return self.fuse(y)


In [9]:
class MobileViT(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )

        self.mv2_block = InvertedResidualBlock(16, 32, stride=1, expand_ratio=2)

        self.mv2_block2 = InvertedResidualBlock(32, 64, stride=2, expand_ratio=2)

        self.mvit_block = MobileViTBlock(
            in_channels=64,
            transformer_dim=96,
            ffn_dim=192,
            n_transformer_blocks=2,
            patch_size=(2, 2)
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.mv2_block(x)
        x = self.mv2_block2(x)
        x = self.mvit_block(x)
        x = self.head(x)
        return x


In [7]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super().__init__()
        hidden_dim = in_channels * expand_ratio
        self.use_res_connect = (stride == 1 and in_channels == out_channels)

        layers = []
        if expand_ratio != 1:
            layers.extend([
                nn.Conv2d(in_channels, hidden_dim, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
            ])
        
        layers.extend([
            nn.Conv2d(hidden_dim, hidden_dim, 3, stride=stride, padding=1, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),
            nn.Conv2d(hidden_dim, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
        ])

        self.block = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.block(x)
        else:
            return self.block(x)


In [10]:
model = MobileViT(num_classes=100)

In [11]:
device = torch.device("mps" if torch.mps.is_available() else "cpu")
print("Using device:", device)

Using device: mps


In [14]:
num_classes = len(base_dataset.classes)  # Automatically detect from dataset
model = MobileViT(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print("classes:", num_classes)

classes: 15


In [53]:
epochs = 10

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    
    train_loss, train_acc = loop_fn('train', train_data, train_loader, model, criterion, optimizer, device)
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    
    val_loss, val_acc = loop_fn('val', val_data, val_loader, model, criterion, optimizer, device)
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")


Epoch 1/10


Train:   0%|          | 0/258 [00:00<?, ?it/s]/Users/anand/Desktop/1mg/repos/thesis/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Train: 100%|██████████| 258/258 [04:01<00:00,  1.07it/s]


Train Loss: 1.5678 | Train Acc: 0.5374


Val: 100%|██████████| 33/33 [01:07<00:00,  2.04s/it]


Val Loss: 1.1911 | Val Acc: 0.6292

Epoch 2/10


...

Val Loss: 1.3502 | Val Acc: 0.5022

Epoch 10/10


Train: 100%|██████████| 258/258 [03:08<00:00,  1.37it/s]


Train Loss: 0.2131 | Train Acc: 0.9374


Val: 100%|██████████| 33/33 [00:11<00:00,  2.77it/s]

Val Loss: 2.1825 | Val Acc: 0.4295


In [ ]:
torch.save(model.state_dict(), "mobilevit_model.pth")

In [54]:
###
# mobile_net solution

model = None
model = torchvision.models.mobilenet_v2(weights="DEFAULT")
# Thay classifier cuối
in_features = model.classifier[1].in_features

print("No of features: ", in_features)
print("Num classes: ", num_classes)
model.classifier[1] = nn.Linear(in_features, num_classes)
model = model.to(device)

for param in model.features.parameters():
    param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

model_name = "mobilenetv2"
num_epochs = 30
train(model_name, num_epochs)


No of features:  1280
Num classes:  15
============================== Epoch 1:


Train: 100%|██████████| 258/258 [00:55<00:00,  4.65it/s]


===Train:	|	Accuracy: 0.7684	|	Loss: 1.0014


Val: 100%|██████████| 2065/2065 [00:19<00:00, 103.67it/s]


===Valid:	|	Accuracy: 0.8978	|	Loss: 0.4875



============================== Epoch 2:


...

===Valid:	|	Accuracy: 0.9511	|	Loss: 0.1444



============================== Epoch 30:


Train: 100%|██████████| 258/258 [00:48<00:00,  5.34it/s]


===Train:	|	Accuracy: 0.9635	|	Loss: 0.1092


Val: 100%|██████████| 2065/2065 [00:19<00:00, 106.62it/s]


===Valid:	|	Accuracy: 0.9554	|	Loss: 0.1454



Best checkpoinpt: 29
Train Accuracy: 0.9642035130224107	|	Train Loss: 0.11098632605995719
Test Accuracy: 0.951089588377724	|	Test Loss: 0.14443045562725174
✅ Accuracy: 0.9554 (1973/2065)


In [55]:
###
# mobile_net solution

model = None
model = torchvision.models.mobilenet_v2(weights="DEFAULT")
# Thay classifier cuối
in_features = model.classifier[1].in_features

print("No of features: ", in_features)
print("Num classes: ", num_classes)
model.classifier[1] = nn.Linear(in_features, num_classes)
model = model.to(device)

for param in model.features.parameters():
    param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

# model_name = "mobilenetv3"
import torchvision.models as models
model_name = models.mobilenet_v3_large(pretrained=True)

num_epochs = 30
train(model_name, num_epochs)


No of features:  1280
Num classes:  15
============================== Epoch 1:


Train: 100%|██████████| 258/258 [00:49<00:00,  5.19it/s]


===Train:	|	Accuracy: 0.7645	|	Loss: 1.0202


Val: 100%|██████████| 2065/2065 [00:20<00:00, 100.61it/s]


===Valid:	|	Accuracy: 0.8891	|	Loss: 0.4966



============================== Epoch 2:


...

===Valid:	|	Accuracy: 0.9530	|	Loss: 0.1444



============================== Epoch 30:


Train: 100%|██████████| 258/258 [00:48<00:00,  5.35it/s]


===Train:	|	Accuracy: 0.9644	|	Loss: 0.1095


Val: 100%|██████████| 2065/2065 [00:19<00:00, 106.27it/s]


===Valid:	|	Accuracy: 0.9516	|	Loss: 0.1448



Best checkpoinpt: 28
Train Accuracy: 0.9650514839491218	|	Train Loss: 0.1120552503362271
Test Accuracy: 0.9559322033898305	|	Test Loss: 0.14128021332451254
✅ Accuracy: 0.9516 (1965/2065)


In [ ]:
###
# mobile_net solution
model =  models.mobilenet_v3_large(pretrained=True)
# Thay classifier cuối
in_features = model.classifier[1].in_features

print("No of features: ", in_features)
print("Num classes: ", num_classes)
model.classifier[1] = nn.Linear(in_features, num_classes)
model = model.to(device)

for param in model.features.parameters():
    param.requires_grad = False


num_epochs = 30
train(model_name, num_epochs, model, train_data, train_loader, test_data, test_loader, criterion, optimizer, device)

AttributeError: 'Hardswish' object has no attribute 'in_features'